In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
card_holders = pd.read_csv('/content/drive/MyDrive/Preppin Data/Data/2024/Flow Card Holders.csv')
non_holders = pd.read_csv('/content/drive/MyDrive/Preppin Data/Data/2024/Non Flow Card Holders.csv')

In [4]:
card_holders.head()

,Date,Flight Number,From,To,Class,Price,Flow Card?,Bags Checked,Meal Type
0,2024-07-22,PA010,Tokyo,New York,Economy,2380.0,Yes,0,Egg Free
1,2024-04-20,PA002,New York,London,Economy,3490.0,Yes,1,Vegan
2,2024-01-23,PA010,Tokyo,New York,Premium Economy,825.0,Yes,1,Vegetarian
3,2024-06-05,PA006,Tokyo,London,First Class,618.0,Yes,3,Vegan
4,2024-03-30,PA004,Perth,London,First Class,446.0,Yes,1,Nut Free


In [5]:
non_holders.head()

,Date,Flight Number,From,To,Class,Price,Flow Card?,Bags Checked,Meal Type
0,2024-02-04,PA004,Perth,London,Economy,1900.0,No,3,NaN
1,2024-09-28,PA008,Perth,New York,Economy,1855.0,No,2,Vegetarian
2,2024-10-01,PA008,Perth,New York,Business Class,634.8,No,0,Vegetarian
3,2024-03-04,PA007,New York,Perth,Business Class,458.4,No,3,Nut Free
4,2024-02-25,PA010,Tokyo,New York,Premium Economy,1435.0,No,0,NaN


In [6]:
# Concatenating the dataframes
df = pd.concat([card_holders, non_holders], axis=0, ignore_index=True)
df

,Date,Flight Number,From,To,Class,Price,Flow Card?,Bags Checked,Meal Type
0,2024-07-22,PA010,Tokyo,New York,Economy,2380.0,Yes,0,Egg Free
1,2024-04-20,PA002,New York,London,Economy,3490.0,Yes,1,Vegan
2,2024-01-23,PA010,Tokyo,New York,Premium Economy,825.0,Yes,1,Vegetarian
3,2024-06-05,PA006,Tokyo,London,First Class,618.0,Yes,3,Vegan
4,2024-03-30,PA004,Perth,London,First Class,446.0,Yes,1,Nut Free
...,...,...,...,...,...,...,...,...,...
3773,2024-03-06,PA006,Tokyo,London,Premium Economy,940.0,No,2,Vegetarian
3774,2024-05-05,PA009,New York,Tokyo,Economy,1360.0,No,3,Nut Free
3775,2024-06-14,PA008,Perth,New York,First Class,245.0,No,1,Dairy Free
3776,2024-01-16,PA010,Tokyo,New York,Economy,2410.0,No,2,Egg Free


In [7]:
# Creating the Quarter field
def define_quarter(x):
  if int(x[5:7])<=3:
    return 1
  elif int(x[5:7])<=6:
    return 2
  elif int(x[5:7])<=9:
    return 3
  else:
    return 4

df['Quarter'] = df['Date'].apply(define_quarter)

In [8]:
# Dropping Date field
df.drop(columns=['Date'], inplace=True)

In [9]:
# Creating aggregations
new_df = df.groupby(['Quarter', 'Flow Card?', 'Class'])\
        .agg({'Price':['median', 'min', 'max']})\
        .rename(columns={'median':'Median_Price', 'min':'Min_Price', 'max':'Max_Price'})\
        .reset_index()

In [10]:
# Renaming the columns
new_df.columns = ['Quarter', 'Flow Card?', 'Class', 'Median_Price', 'Min_Price', 'Max_Price']

In [11]:
# Creating separate flows
median_df = new_df[['Quarter', 'Flow Card?', 'Class', 'Median_Price']]
min_df = new_df[['Quarter', 'Flow Card?', 'Class', 'Min_Price']]
max_df = new_df[['Quarter', 'Flow Card?', 'Class', 'Max_Price']]

In [12]:
# Pivoting the three flows
median_pivot_df = median_df.pivot_table(index=['Quarter', 'Flow Card?'], columns=['Class'], values=['Median_Price']).reset_index()
median_pivot_df.columns = ['Quarter', 'Flow Card?', 'Premium Economy', 'First Class', 'Economy', 'Business Class']

min_pivot_df = min_df.pivot_table(index=['Quarter', 'Flow Card?'], columns=['Class'], values=['Min_Price']).reset_index()
min_pivot_df.columns = ['Quarter', 'Flow Card?', 'Premium Economy', 'First Class', 'Economy', 'Business Class']

max_pivot_df = max_df.pivot_table(index=['Quarter', 'Flow Card?'], columns=['Class'], values=['Max_Price']).reset_index()
max_pivot_df.columns = ['Quarter', 'Flow Card?', 'Premium Economy', 'First Class', 'Economy', 'Business Class']

In [13]:
# Concatenating the three flows
output_df = pd.concat([median_pivot_df, min_pivot_df, max_pivot_df], axis=0)

In [14]:
# Renaming the columns
output_df = output_df[['Flow Card?', 'Quarter', 'Economy', 'Premium Economy', 'Business Class', 'First Class']]
output_df.columns = ['Flow Card?', 'Quarter', 'Economy', 'Premium', 'Business', 'First']

In [15]:
output_df.head()

,Flow Card?,Quarter,Economy,Premium,Business,First
0,No,1,438.0,574.8,1075.00,2340.0
1,Yes,1,447.5,523.2,1160.00,2325.0
2,No,2,445.0,553.8,1205.00,2325.0
3,Yes,2,459.0,517.8,1071.25,2290.0
4,No,3,487.0,490.8,1125.00,2285.0


In [16]:
# Exporting the output
output_df.to_csv('Week_2_Output.csv')